# Imports

In [ ]:
%cd /content
!git clone https://github.com/timojl/clipseg.git

/content
Cloning into 'clipseg'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 226 (delta 66), reused 51 (delta 51), pack-reused 141
Receiving objects: 100% (226/226), 1.40 MiB | 6.11 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [ ]:
# Commits on Sep 27, 2022
%cd /content/clipseg
!git checkout 515ca6ec2d066d447240c1dd79f3bbbee685bd29

/content/clipseg
Note: switching to '515ca6ec2d066d447240c1dd79f3bbbee685bd29'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 515ca6e added samples for fine-grained weights


In [ ]:
!pip install git+https://github.com/openai/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1

%cd /content/clipseg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git (to revision d50d76daa670286dd6cacf3bcd80b5e4823fc8e1) to /tmp/pip-req-build-m36cimab
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-m36cimab
  Running command git rev-parse -q --verify 'sha^d50d76daa670286dd6cacf3bcd80b5e4823fc8e1'
  Running command git fetch -q https://github.com/openai/CLIP.git d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Running command git checkout -q d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369398 sha256=c158305ee46998a6b0c2f55d7fbaddc18cca19e62db587f84c885a462b16ed91
  Stored in dir

In [ ]:
import os

import torch
import requests

from models.clipseg import CLIPDensePredT
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt
import cv2
import numpy as np
import PIL

device = 'cuda' if torch.cuda.is_available() else "cpu"
print("using device is", device)

using device is cuda


In [ ]:
!mkdir pretrained

if not os.path.exists('pretrained/weights.zip'):
  !wget https://owncloud.gwdg.de/index.php/s/ioHbRzFx6th32hn/download -O pretrained/weights.zip
  !unzip -d pretrained/weights -j pretrained/weights.zip

--2023-05-07 09:34:17--  https://owncloud.gwdg.de/index.php/s/ioHbRzFx6th32hn/download
Resolving owncloud.gwdg.de (owncloud.gwdg.de)... 134.76.23.45
Connecting to owncloud.gwdg.de (owncloud.gwdg.de)|134.76.23.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘pretrained/weights.zip’

pretrained/weights.     [              <=>   ]   9.68M  3.51MB/s    in 2.8s    

2023-05-07 09:34:21 (3.51 MB/s) - ‘pretrained/weights.zip’ saved [10146905]

Archive:  pretrained/weights.zip
 extracting: pretrained/weights/rd16-uni.pth  
 extracting: pretrained/weights/rd64-uni-refined.pth  
 extracting: pretrained/weights/rd64-uni.pth  


In [ ]:
# load model
model = CLIPDensePredT(version='ViT-B/16', reduce_dim=64)
model.eval();

model.load_state_dict(torch.load('pretrained/weights/rd64-uni.pth', map_location=torch.device(device)), strict=False);

100%|████████████████████████████████████████| 335M/335M [00:02<00:00, 160MiB/s]


# Mask Manipulation function


In [ ]:
from copy import deepcopy 
'''
function to adjust the noise level for a given mask
input_mask: the 512x512 uint mask matrix
noise_level: a float value between 0 and 1, represents how many noise to add according to number of value 255 inside the given mask (masked pixels)
mode: 0 or 1 or 2
      0 - only change originally masked pixels
      1 - only change originally not masked pixels (across the entire image)
      2 - change both masked and un masksed pixels (across the entire image)
'''
def add_noise_to_mask(input_mask, noise_level, mode=0):
  output_mask = deepcopy(input_mask)
  num_masked = np.count_nonzero(input_mask == 255)
  masked_indices = np.asarray(np.where(input_mask == 255)).T
  background_indices = np.asarray(np.where(input_mask == 0)).T
  num_noise = int(noise_level*num_masked)

  if mode == 0:
    change_indices =[masked_indices[x] for x in np.random.choice(range(len(masked_indices)), size=int(noise_level*num_masked), replace=False)]
    for x in change_indices:
      output_mask[x[0]][x[1]] = 0
  elif mode == 1:
    change_indices =[background_indices[x] for x in np.random.choice(range(len(background_indices)), size=int(noise_level*num_masked), replace=False)]
    for x in change_indices:
      output_mask[x[0]][x[1]] = 255
  else:
    change_prob = float(noise_level*num_masked)/(512.0*512.0)
    change_mask = np.random.choice([0, 255], size=(512,512), p=[1-change_prob, change_prob])
    output_mask = np.add(output_mask, change_mask)
    output_mask[output_mask==510] = 0
    
  output_mask = np.uint8(output_mask)
  return output_mask



# image processing functions

In [ ]:
import requests
from io import BytesIO

def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")

def norm_photo(photo):
  transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((512, 512)),])
  photo = transform(photo).unsqueeze(0)
  return photo

In [ ]:
# single image
noise_levels = [0.0001, 0.001, 0.01, 0.1]
total_experiments = len(noise_levels)*3
curr_experiments = 1


seg_prompts = 'white horse'
input_image = PIL.Image.open('example_image_2.jpg').resize((512,512))
img = norm_photo(input_image)

# save normalized input
filename = "input.jpg"
plt.imsave(filename, input_image)
del filename

# generate mask
with torch.no_grad():
    preds = model(img.repeat(len(seg_prompts),1,1,1), seg_prompts)[0]
filename = "ori_mask.jpg"
plt.imsave(filename,torch.sigmoid(preds[0][0]))
img2 = cv2.imread(filename)
gray_image = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
del img2
(thresh, bw_image) = cv2.threshold(gray_image, 100, 255, cv2.THRESH_BINARY)

# mask array
mask_arr = np.uint8(bw_image)

# save original mask
cv2.cvtColor(bw_image, cv2.COLOR_BGR2RGB)
original_mask = Image.fromarray(np.uint8(bw_image)).convert('RGB').resize((512, 512))
original_mask.save(filename)
del filename

# generate mask for all noise levels and modes
for n,noise in enumerate(noise_levels):
  for m in range(3):
    # generate new mask
    curr_mask = add_noise_to_mask(mask_arr, noise, mode=m)

    # save new mask
    curr_mask_img = Image.fromarray(curr_mask).convert('RGB')
    filename = "mask_mode"+str(m)+"_noise"+str(n)+".jpg"
    curr_mask_img.save(filename)

    # print progress
    print("current progress: %d/%d, %s" % (curr_experiments, total_experiments, filename))
    curr_experiments += 1

    del curr_mask
    del curr_mask_img
    del filename

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


current progress: 1/12, mask_mode0_noise0.jpg
current progress: 2/12, mask_mode1_noise0.jpg
current progress: 3/12, mask_mode2_noise0.jpg
current progress: 4/12, mask_mode0_noise1.jpg
current progress: 5/12, mask_mode1_noise1.jpg
current progress: 6/12, mask_mode2_noise1.jpg
current progress: 7/12, mask_mode0_noise2.jpg
current progress: 8/12, mask_mode1_noise2.jpg
current progress: 9/12, mask_mode2_noise2.jpg
current progress: 10/12, mask_mode0_noise3.jpg
current progress: 11/12, mask_mode1_noise3.jpg
current progress: 12/12, mask_mode2_noise3.jpg


# Data

In [ ]:
urls = ['http://farm6.staticflickr.com/5043/5306210272_5ae5fe553e_z.jpg',
'http://farm6.staticflickr.com/5060/5514795117_aba57ba82f_z.jpg',
'http://farm7.staticflickr.com/6228/6242985772_7253a1e774_z.jpg',
'http://farm1.staticflickr.com/24/35275654_7d9397809b_z.jpg',
'http://farm6.staticflickr.com/5107/5636416098_bfc492a357_z.jpg',
'http://farm3.staticflickr.com/2823/9222897874_93f59af0a6_z.jpg',
'http://farm3.staticflickr.com/2094/2435518227_10ab34b7c1_z.jpg',
'http://farm7.staticflickr.com/6096/6287255884_22514e5d17_z.jpg',
'http://farm7.staticflickr.com/6028/5949876155_74817efc59_z.jpg',
'http://farm5.staticflickr.com/4045/4212548498_1f8c8110aa_z.jpg',
'http://farm4.staticflickr.com/3454/3356237814_518e49c795_z.jpg',
'http://farm6.staticflickr.com/5452/9205515621_4ce6319d66_z.jpg',
'http://farm4.staticflickr.com/3504/3700248299_89346cf5ae_z.jpg',
'http://farm8.staticflickr.com/7206/7094512025_391c37f4df_z.jpg',
'http://farm9.staticflickr.com/8075/8324011371_68fca0267c_z.jpg',
'http://farm1.staticflickr.com/38/116015759_cbabdbf4d6_z.jpg',
'http://farm4.staticflickr.com/3118/2296947987_266e04ea63_z.jpg',
'http://farm1.staticflickr.com/157/439127630_7deea02049_z.jpg',
'http://farm5.staticflickr.com/4017/4515100161_f9956e4b58_z.jpg',
'http://farm2.staticflickr.com/1289/766387238_331186506a_z.jpg',
'http://farm3.staticflickr.com/2531/3797908806_a0381dcc3b_z.jpg',
'http://farm9.staticflickr.com/8363/8335892848_8280029ef5_z.jpg',
'http://farm1.staticflickr.com/104/281539229_c54f5c027a_z.jpg',
'http://farm1.staticflickr.com/157/336331930_9be45527ac_z.jpg',
'http://farm4.staticflickr.com/3034/2570055632_3d9e81ce35_z.jpg']

prompts = ["elephant", "clock", "husky", "laptop computer", "black and white bird",
           "person","yellow fire hydrant", "airplane", "small plane","person", "suitcase",
           "yellow rose","woman","orange and purple tulip", "white microwave", "microwave oven", "sandwich", "cat",
           "strawberries", "hot dog", "backpack", "bed", "suitcase", "wii remote", "couch"]

re_prompts = ["tiger", "window", "cat", "stack of books", "squirrel",
           "black and white mickey mouse","metal dustbin", "fighter jet", "house","dog", "gray computer",
           "pink tulip","monkey","apple", "white suitcase", "tv", "omelette", "cabybara",
           "oranges", "cookies", "dog", "pool", "stack of books", "magic wand", "pile of rocks"]

'''
full_prompts = ["an tiger walking through a forested area",
              "a window that is next to an eagle statue",
              "a large cat sitting and waiting for a treat",
              "a stack of books sitting on top of a table",
              "a squirrel sitting in an evergreen tree",
              "a black and white mickey mouse jumping a skate board in the air",
              "a metal dustbin on the side of a street",
              "an fighter jet parked in a spot at the airport",
              "A house that is sitting on a runway",
              "a dog entering a crosswalk along a city street",
              "a gray computer sitting on the floor ",
              "a vase sitting in the sun holding a pink tulip",
              "a monkey opening the doors from a bathroom",
              "a vase of apple on a wood table",
              "a kitchen with a white suitacase oven sitting on top of a counter",
              "a tv on top of a stove",
              "a bowl of soup sitting on a plate with omelette and spoon near a basket of crackers",
              "a gray cabybara lying on a carpet in a room",
              "two bagels next to three oranges on a white plate",
              "a couple of cookies sitting on a blue plate on a computer desk",
              "a dog sits on a small bed in a small room",
              "a pool is on the floor on wooden slats",
              "there is a stack of books laid out in front of a couch",
              "a girl standing up with an arm outstretched and a magic wand in hand",
              "a laptop sitting on a pile of rocks next to a remote control"]
'''

noise_levels = [0.001, 0.1]

# Generate Segmentation and Export mask photoes

In [ ]:
total_experiments = len(noise_levels)*len(urls)*3
curr_experiments = 1

for url_i,url in enumerate(urls):
  seg_prompts = prompts[url_i]
  input_image = download_image(url).resize((512, 512))
  img = norm_photo(input_image)

  # save normalized input
  filename = "input_"+str(url_i)+".jpg"
  plt.imsave(filename, img)
  del filename

  # generate mask
  with torch.no_grad():
      preds = model(img.repeat(len(seg_prompts),1,1,1), seg_prompts)[0]
  filename = "input_mask_"+str(url_i)+".jpg"
  plt.imsave(filename,torch.sigmoid(preds[0][0]))
  img2 = cv2.imread(filename)
  gray_image = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
  del img2
  (thresh, bw_image) = cv2.threshold(gray_image, 100, 255, cv2.THRESH_BINARY)

  # mask array
  mask_arr = np.uint8(bw_image)

  # save original mask
  cv2.cvtColor(bw_image, cv2.COLOR_BGR2RGB)
  original_mask = Image.fromarray(np.uint8(bw_image)).convert('RGB').resize((512, 512))
  original_mask.save(filename)
  del filename

  # generate mask for all noise levels and modes
  for n,noise in enumerate(noise_levels):
    for m in range(3):
      # generate new mask
      curr_mask = add_noise_to_mask(mask_arr, noise, mode=m)

      # save new mask
      curr_mask_img = Image.fromarray(curr_mask).convert('RGB')
      filename = "edit_mask_"+str(url_i)+"_mode"+str(m)+"_noise"+str(n)+".jpg"
      curr_mask_img.save(filename)

      # print progress
      print("current progress: %d/%d, %s" % (curr_experiments, total_experiments, filename))
      curr_experiments += 1

      del curr_mask
      del curr_mask_img
      del filename

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


current progress: 1/150, edit_mask_0_mode0_noise0.jpg
current progress: 2/150, edit_mask_0_mode1_noise0.jpg
current progress: 3/150, edit_mask_0_mode2_noise0.jpg
current progress: 4/150, edit_mask_0_mode0_noise1.jpg
current progress: 5/150, edit_mask_0_mode1_noise1.jpg
current progress: 6/150, edit_mask_0_mode2_noise1.jpg
current progress: 7/150, edit_mask_1_mode0_noise0.jpg
current progress: 8/150, edit_mask_1_mode1_noise0.jpg
current progress: 9/150, edit_mask_1_mode2_noise0.jpg
current progress: 10/150, edit_mask_1_mode0_noise1.jpg
current progress: 11/150, edit_mask_1_mode1_noise1.jpg
current progress: 12/150, edit_mask_1_mode2_noise1.jpg
current progress: 13/150, edit_mask_2_mode0_noise0.jpg
current progress: 14/150, edit_mask_2_mode1_noise0.jpg
current progress: 15/150, edit_mask_2_mode2_noise0.jpg
current progress: 16/150, edit_mask_2_mode0_noise1.jpg
current progress: 17/150, edit_mask_2_mode1_noise1.jpg
current progress: 18/150, edit_mask_2_mode2_noise1.jpg
current progress: 1